In [1]:
import kfp, os
import kfp.dsl as dsl 
import kfp.compiler as compiler
import kubernetes.client.models as k8s
import namesgenerator
from utils import *

In [2]:
NAMESPACE = os.environ.get("NAMESPACE") 

In [3]:
kubeflow_address = f"http://{NAMESPACE}.kubeflow.odsc.k8s.hydrosphere.io"
hydrosphere_address = f"http://{NAMESPACE}.serving.odsc.k8s.hydrosphere.io"

# Download

### Pipeline

At this stage we will obtain all training data for our pipeline.

Create Kubernetes PVC resource

In [6]:
storage_pvc = k8s.V1PersistentVolumeClaimVolumeSource(claim_name="storage")
storage_volume = k8s.V1Volume(name="storage", persistent_volume_claim=storage_pvc)
storage_volume_mount = k8s.V1VolumeMount(mount_path="{{workflow.parameters.mount-path}}", name="storage")

Define container operation

In [7]:
username = "tidylobster"

In [8]:
def download_op(**kwargs):
    download = dsl.ContainerOp(
        name="download",
        image=f"{username}/mnist-pipeline-download:latest",
        file_outputs={"data_path": "/data_path.txt"},
        arguments=[
            "--mount-path", kwargs["mount_path"]
        ],
    )
    
    download.add_volume(storage_volume)
    download.add_volume_mount(storage_volume_mount)
    return download

In [9]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
):
    
    download = download_op(
        mount_path=mount_path)

In [10]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [11]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [11]:
# Create Pipelines client
client = kfp.Client(kubeflow_address)

In [12]:
# Define an experiment name
experiment_name='MNIST Showreal'

In [13]:
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id

In [14]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(experiment_id, run_name, "pipeline.yaml")

Starting a new run with the name amazing_leavitt


# Train

### Pipeline

At this stage we will create a model & train it on the downloaded data.

Define container operation

In [16]:
def train_op(download, **kwargs):
    train = dsl.ContainerOp(
        name="train",
        image=f"{username}/mnist-pipeline-train:latest",
        file_outputs={"accuracy": "/accuracy.txt"},
        arguments=[
            "--data-path", download.outputs["data_path"], 
            "--mount-path", kwargs["mount_path"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"]
        ],
    )

    train.add_volume(storage_volume)
    train.add_volume_mount(storage_volume_mount)
    return train

In [17]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')

In [18]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [19]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [34]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10"
    }
)

Starting a new run with the name romantic_albattani


# Release

### Pipeline

At this stage we will release the trained model to the cluster

Define container operation

In [21]:
def release_op(download, train, **kwargs):
    release = dsl.ContainerOp(
        name="release",
        image=f"{username}/mnist-pipeline-release:latest",
        file_outputs={"model-version": "/model-version.txt"},
        arguments=[
            "--data-path", download.outputs["data_path"],
            "--mount-path", kwargs["mount_path"],
            "--model-name", kwargs["model_name"],
            "--accuracy", train.outputs["accuracy"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"],
        ],
    )

    release.add_volume(storage_volume) 
    release.add_volume_mount(storage_volume_mount)
    return release

In [22]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address=""
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train,
        mount_path=mount_path,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)

In [23]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [24]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [28]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name tender_shaw


# Deploy to Stage

### Pipeline

At this stage we are deploying the model to the stage application to run integration tests.

Define container operation

In [26]:
def deploy_to_stage_op(release, **kwargs):
    deploy_to_stage = dsl.ContainerOp(
        name="deploy_to_stage",
        image=f"{username}/mnist-pipeline-deploy-to-stage:latest",
        file_outputs={"stage-app-name": "/stage-app-name.txt"},
        arguments=[
            "--model-version", release.outputs["model-version"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--model-name", kwargs["model_name"],
        ],
    )
    
    return deploy_to_stage

In [27]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train,
        mount_path=mount_path,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release,
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)


In [28]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [29]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [41]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name awesome_austin


# Test

### Pipeline

At this stage we are performing integration tests.

Define container operation

In [31]:
def test_op(download, stage, **kwargs):
    test = dsl.ContainerOp(
        name="test",
        image=f"{username}/mnist-pipeline-test:latest", 
        arguments=[
            "--stage-app-name", stage.outputs["stage-app-name"], 
            "--data-path", download.outputs["data_path"],
            "--mount-path", kwargs["mount_path"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--acceptable-accuracy", kwargs["acceptable_accuracy"],
            "--model-name", kwargs["model_name"], 
        ],
    )

    test.add_volume(storage_volume) 
    test.add_volume_mount(storage_volume_mount)
    return test

In [32]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    acceptable_accuracy="0.90",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train,
        mount_path=mount_path,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release,
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        download, 
        deploy_to_stage,
        mount_path=mount_path,
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)

In [33]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [34]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [47]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name suspicious_jepsen


# Deploy to Production

### Pipeline

And finally deploying the model to production application

Define container operation

In [36]:
def deploy_to_prod_op(release, **kwargs):
    deploy_to_prod = dsl.ContainerOp(
        name="deploy_to_prod",
        image=f"{username}/mnist-pipeline-deploy-to-prod:latest",
        arguments=[
            "--model-version", release.outputs["model-version"],
            "--model-name", kwargs["model_name"],
            "--hydrosphere-address", kwargs["hydrosphere_address"]
        ],
    )

    return deploy_to_prod

In [37]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    acceptable_accuracy="0.90",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train,
        mount_path=mount_path,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release,
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        download, 
        deploy_to_stage,
        mount_path=mount_path,
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)
    
    deploy_to_prod = deploy_to_prod_op(
        release,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address)
    deploy_to_prod.after(test)

In [38]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [39]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [40]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name quizzical_dubinsky


## Production traffic

In [ ]:
simulate_production_traffic(shuffle=False)

# Sample

### Pipeline

Once we have a fully functional pipeline, we would like to automate running this pipeline.

Define container operation

In [43]:
def sample_op(**kwargs):
    sample = dsl.ContainerOp(
        name="sample",
        image=f"{username}/mnist-pipeline-sample:latest", 
        file_outputs={"data_path": "/data_path.txt"},
        arguments=[
            "--mount-path", kwargs["mount_path"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
        ]
    )  
    
    sample.add_volume(storage_volume)
    sample.add_volume_mount(storage_volume_mount)
    return sample

In [50]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    acceptable_accuracy="0.90",
):
    
    sample = sample_op(
        mount_path=mount_path, 
        hydrosphere_address=hydrosphere_address)
    
    train = train_op(
        sample, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(sample)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        sample, 
        train,
        mount_path=mount_path, 
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release,
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        sample, 
        deploy_to_stage,
        mount_path=mount_path,
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)
    
    deploy_to_prod = deploy_to_prod_op(
        release,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address)
    deploy_to_prod.after(test)

In [51]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [52]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test

In [53]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.30",
    }
)

Starting a new run with the name serene_perlman
